# <p style="text-align: center;">Harmonizing Multi-Site Structural MRI Datasets</p>

### <p style="text-align: center;">David MacDonald</p>

# Plan
- Start simple.
- Add to the presentation every day.
- Look at other presentations every day. Brainhack ones would be a good start.
- Make it better over the course of a week.
- For now, can use canva for graphics.
- OR, LEARN to use GitPitch, which has more layout tools, I think. It was pretty cool.

- Try to describe my workflow using freepik and/or canva for images, icons, arrows, etc.
- Get icons for all of the tools that I'm using

# To Do
 * Fix the Purpose section. It's not clear what's my BHS project and what's not. Maybe just a slide before describing my own research, then on to purpose.

## Multi-site MRI


 * Neuroimaging data, such as MRI, is time-consuming and very costly to collect, but rich in information.
 * Multi-site, open data allows for much larger sample sizes and much greater statistical power in MRI studies, while distributing the cost across many research centers.



### <p style="text-align: center;">HOWEVER . . .</p>

 ### <p style="text-align: center;">HOWEVER . . .</p>
 
 * Differences between scans due to technical factors can overwhelm those due to biological factors
 ### <p style="text-align: left;">This introduces VARIABILITY, and . . . </p>

  

 
 
 
 * Different sites likely sample different populations
 
 ### <p style="text-align: left;">Which introduces BIAS. . . </p>

## Purpose

### The Context
Examining the volume of thalamus, globus pallidus, striatum in ASD, using the [ABIDE](http://fcon_1000.projects.nitrc.org/indi/abide/) dataset

### Multi-Site Data Harmonization
 * Use linear mixed-effect models.
 * Use the open source [neuroCombat](https://github.com/Jfortin1/ComBatHarmonization) library, which was originally intended to mitigate batch effects in genetic studies, but has since been adapted to work with neuroimaging data.


## My (Relevant) Background

* 1st year M.Sc. student at McGill University
* Using structural MRI to study subcortical structures in Autism Spectrum Disorders

* Prior neuroscience work in a wet lab: IHC and molecular
* Experience using Excel, SPSS point-and-click interface

* Some experience with:
    * bash
    * programming (learning R, C++ a *long* time ago)
    * HPC through Compute Canada

* Minimal experience with:
    * git and github


## Learning Goals
 * Learn:
     * Git and GitHub
     * Python, including data manipulation and visualization
        * Python environment virtualization: conda and/or venv
     * Jupyter notebooks
     * [ComBat](https://github.com/Jfortin1/ComBatHarmonization)

## ComBat
$$y_{r,i,j} = \alpha_r + X_{i,j} \beta_r + \gamma_{r,i} + \delta_{r,i} \epsilon_{r,i,j}$$

$$\epsilon_{r,i,j} = \frac{y_{r,i,j} - \alpha_r - X_{i,j} \beta_r - \gamma_{r,i}}{\delta_{r,i}}$$

$$y_{r,i,j}^{ComBat} = \alpha_r + X_{i,j} \beta_r + \epsilon_{r,i,j}$$


<center><img src="slide_test2.png" alt="Image" style="width: 1500px" align="center"/></center>

# Workflow Section

# Results Section

# Conclusion Section

## Open Science Tools

 * Git and GitHub

 * bash scripting
 * R
 * Python
     * pandas for data manipulation
     * seaborn for data visualization
     * ipywidgets for interactive data visualization
     * statsmodels for data analysis

 * Jupyter Notebooks
     * Presentations with RISE

 * Virtualization with conda
 * Containerization with Docker
     * Docker Hub ?????????????????????????????????????????????????

## Skills and Technologies

 * Linear mixed-effects modelling with statsmodels in python
 * [ComBat](https://github.com/Jfortin1/ComBatHarmonization) for harmonizing neuroimaging data
 
 Both were learned independently during BHS, with support of instructors and peers

## Relevance

 * With the increasing use of open datasets in neuroimaging, it is important to have tools to manage non-biological sources of variation in scans
 * ComBat is used extensively in genetics, relatively new in neuroimaging
     * Has been applied to DTI, fMRI, CT data, but not to subcortical volume / morphometry
 * Uses entirely open science software and open data

## Further Work

 * Contribute missing features to the Python version of ComBat on GitHub
     * Handling missing values
     * Empirical Bayes: visualizing parameter distributions
 * Compare to meta-analytic combination of single-site data
 * Improve visualizations
 * Determine whether using empirical Bayes improves the analysis for this type of data (low number of features)

## Challenges

 * Python version of ComBat does not handle missing values
     * Chose to use R version
     * Planned to use rpy2
 * Problems with setting up reproducible environment for R
     * Could not virtualize R environment
     * Version conflicts with required packages
     * Conda interacted with R environment
 * Docker setup required manual editing of neuroDocker file
     * No problem! BHS presentation was excellent resource
 * Jupyter notebook can disconnect from server, meaning that saving fails until you reconnect by finding and entering the correct token. Lost several hours of work in Python.

## Unexpected Benefits

 * Created a toy pipeline using multiple tools and bash scripting
 * Used Docker to containerize the pipeline
 * Interactive visualization of data
 * 